In [26]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["do_an_tot_nghiep"]

myCar = mydb["cars"]
mySpec = mydb["specs"]
myQuery = {'nameCar': '2025 BMW X5'}

if not myCar.find_one(myQuery):
    print('Không tìm thấy dữ liệu')
    myCar.insert_one(myQuery)
    print('Đã thêm dữ liệu')

x =  myCar.find_one(myQuery)
print(x)



{'_id': ObjectId('6633a596bfeb4f0a77e3d37c'), 'nameCar': '2025 BMW X5'}


In [4]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["do_an_tot_nghiep"]

print(mydb.list_collection_names())


['specs', 'relatedInfos', 'users', 'cars', 'images']


In [9]:
###############  Thêm tất cả spec trừ safety và not table vào collection specs  ####################

import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Improved error handling and logging
import logging

# Connect to MongoDB
try:
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = client["do_an_tot_nghiep"]
except pymongo.errors.ConnectionFailure as e:
    logging.error("Error connecting to MongoDB:", e)
    exit(1)

# Collections
myCar = mydb["cars"]
mySpec = mydb["specs"]

# Load data from Excel (assuming consistent sheet name)
try:
    file_path = 'F:\Projects\Python\Scraper_data\BMW_Car_data.xlsx'
    df = pd.read_excel(file_path, sheet_name='Car')
    link_car_values = df['Link'].tolist()
    name_car_values = df['Name Car'].tolist()
except FileNotFoundError as e:
    logging.error("Error reading Excel file:", e)
    exit(1)

for nameCar, linkCar in zip(name_car_values, link_car_values):
    # Check if car exists in 'cars' collection (avoid duplicate inserts)
    carQuery = {'nameCar': nameCar}
    x = myCar.find_one(carQuery)

    if not x:
        print('Không tìm thấy dữ liệu xe:', nameCar)
        myCar.insert_one(carQuery)
        print('Đã thêm dữ liệu xe:', nameCar)
        x = myCar.find_one(carQuery)

    # Make request and parse HTML
    try:
        page = requests.get(linkCar)
        soup = BeautifulSoup(page.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        logging.error("Error fetching webpage:", e)
        continue  # Skip to next iteration on fetching error

    # Target tables based on class (consider more robust selectors)
    tables = soup.find_all('table', class_='css-2a9fnn')

    # Iterate over targeted tables and predefined titles
    titles = ["Dimensions, Weights & Capacities", "Exterior", "Fuel Economy", "Mechanical", "Performance", "Warranty"]
    for title, product in zip(titles, tables):
        # Extract specification data
        specNames = product.select('.css-1kws18 div')
        values = product.select('.css-irk93x div')
        step = 0
        # Store extracted data with appropriate structure
        for specName, value in zip(specNames, values):
            specQuery = {'category': title.strip(), "specName": specName.text.strip(), "idCar": x["_id"]}
            mySpecFind = mySpec.find_one(specQuery)

            if mySpecFind:
                print('Dữ liệu đã tồn tại:', nameCar)
                step = 1
                break
            Spec = {
                "idCar": x["_id"],
                "category": title.strip(),  # Ensure cleaned category name
                "specName": specName.text.strip(),
                "value": value.text.strip(),
            }
            mySpec.insert_one(Spec)
        print(step)
        if step == 1:
            step = 0
            break
    print('open ', step)

    print('Đã thêm dữ liệu thông số cho xe:', nameCar)

print('Đã hoàn tất thêm dữ liệu vào collection specs')


0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW X5
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW X3
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 5 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 3 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 7 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 6 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW Z4
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW M3
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW X5
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW X3
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 5 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 3 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 7 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 6 Series
0
0
0
0
0
0
open  0
Đã thêm dữ liệu thông 

In [10]:
###############  Thêm tất cả spec not table vào collection specs  ####################

import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Improved error handling and logging
import logging

# Connect to MongoDB
try:
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = client["do_an_tot_nghiep"]
except pymongo.errors.ConnectionFailure as e:
    logging.error("Error connecting to MongoDB:", e)
    exit(1)

# Collections
myCar = mydb["cars"]
mySpec = mydb["specs"]

# Load data from Excel (assuming consistent sheet name)
try:
    file_path = 'F:\Projects\Python\Scraper_data\BMW_Car_data.xlsx'
    df = pd.read_excel(file_path, sheet_name='Car')
    link_car_values = df['Link'].tolist()
    name_car_values = df['Name Car'].tolist()
except FileNotFoundError as e:
    logging.error("Error reading Excel file:", e)
    exit(1)

for nameCar, linkCar in zip(name_car_values, link_car_values):
    # Check if car exists in 'cars' collection (avoid duplicate inserts)
    carQuery = {'nameCar': nameCar}
    x = myCar.find_one(carQuery)

    if not x:
        print('Không tìm thấy dữ liệu xe:', nameCar)
        myCar.insert_one(carQuery)
        print('Đã thêm dữ liệu xe:', nameCar)
        x = myCar.find_one(carQuery)

    # Make request and parse HTML
    try:
        page = requests.get(linkCar)
        soup = BeautifulSoup(page.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        logging.error("Error fetching webpage:", e)
        continue  # Skip to next iteration on fetching error

    # Extract and store tables
    products = soup.select('div.css-1cu80qq:nth-of-type(n+7) div.css-1sgsyzb')

    for product in products:
        # Find the h3 tag within the product
        category = product.select('h3')
        specs = product.select('.css-b4wpih span')
        step = 0
        # Store extracted data with appropriate structure
        for spec in specs:
            specQuery = {'category': category[0].text, "specName": spec.text.strip(), "idCar": x["_id"]}
            mySpecFind = mySpec.find_one(specQuery)

            if mySpecFind:
                print('Dữ liệu đã tồn tại:', nameCar)
                step = 1
                break
            Spec = {
                "idCar": x["_id"],
                "category": category[0].text,  # Ensure cleaned category name
                "specName": spec.text.strip(),
            }
            mySpec.insert_one(Spec)
        print(step)
        if step == 1:
            step = 0
            break
    print('open ', step)

    print('Đã thêm dữ liệu thông số cho xe:', nameCar)

print('Đã hoàn tất thêm dữ liệu vào collection specs')


0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW X5
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW X3
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 5 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 3 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 7 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 6 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW Z4
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW M3
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW X5
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW X3
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 5 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 3 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 7 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW 6 Series
0
0
0
0
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW Z4
0
0
0
0
open  0
Đã thêm dữ liệu thông

In [ ]:
###############  Thêm tất cả spec safety vào collection specs  ####################

import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Improved error handling and logging
import logging

# Connect to MongoDB
try:
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = client["do_an_tot_nghiep"]
except pymongo.errors.ConnectionFailure as e:
    logging.error("Error connecting to MongoDB:", e)
    exit(1)

# Collections
myCar = mydb["cars"]
mySpec = mydb["specs"]

# Load data from Excel (assuming consistent sheet name)
try:
    file_path = 'F:\Projects\Python\Scraper_data\BMW_Car_data.xlsx'
    df = pd.read_excel(file_path, sheet_name='Car')
    link_car_values = df['Link'].tolist()
    name_car_values = df['Name Car'].tolist()
except FileNotFoundError as e:
    logging.error("Error reading Excel file:", e)
    exit(1)

for nameCar, linkCar in zip(name_car_values, link_car_values):
    # Check if car exists in 'cars' collection (avoid duplicate inserts)
    carQuery = {'nameCar': nameCar}
    x = myCar.find_one(carQuery)

    if not x:
        print('Không tìm thấy dữ liệu xe:', nameCar)
        myCar.insert_one(carQuery)
        print('Đã thêm dữ liệu xe:', nameCar)
        x = myCar.find_one(carQuery)

    # Make request and parse HTML
    try:
        page = requests.get(linkCar)
        soup = BeautifulSoup(page.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        logging.error("Error fetching webpage:", e)
        continue  # Skip to next iteration on fetching error

    # Extract and store tables
    products = soup.select('.css-vqoa8l div.css-1044rcd')
    step = 0
    for product in products:
        # Find the h3 tag within the product
        specs = product.select('.css-1pwofzj strong')
        specName = specs[0].text
        values = product.select('.css-11jj7n3 p')
        step = 0
        # Store extracted data with appropriate structure
        for value in values:
            specQuery = {'category': "Safety Technology", "specName": specName, "idCar": x["_id"]}
            mySpecFind = mySpec.find_one(specQuery)

            if mySpecFind:
                print('Dữ liệu đã tồn tại:', nameCar)
                step = 1
                break
            Spec = {
                "idCar": x["_id"],
                "category": "Safety Technology",  # Ensure cleaned category name
                "specName": specName,
                "value" : value.text.strip()
            }
            mySpec.insert_one(Spec)
        print(step)
        if step == 1:
            step = 0
            break
    print('open ', step)

    print('Đã thêm dữ liệu thông số cho xe:', nameCar)

print('Đã hoàn tất thêm dữ liệu vào collection specs')


In [5]:
###############  Thêm tất cả related info vào collection specs  ####################

import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Improved error handling and logging
import logging

# Connect to MongoDB
try:
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = client["do_an_tot_nghiep"]
except pymongo.errors.ConnectionFailure as e:
    logging.error("Error connecting to MongoDB:", e)
    exit(1)

# Collections
myCar = mydb["cars"]
myRelatedInfo = mydb["relatedInfos"]

# Load data from Excel (assuming consistent sheet name)
try:
    file_path = 'F:\Projects\Python\Scraper_data\BMW_Car_data.xlsx'
    df = pd.read_excel(file_path, sheet_name='Car')
    link_car_values = df['Link'].tolist()
    name_car_values = df['Name Car'].tolist()
except FileNotFoundError as e:
    logging.error("Error reading Excel file:", e)
    exit(1)

for nameCar, linkCar in zip(name_car_values, link_car_values):
    # Check if car exists in 'cars' collection (avoid duplicate inserts)
    carQuery = {'nameCar': nameCar}
    x = myCar.find_one(carQuery)

    if not x:
        print('Không tìm thấy dữ liệu xe:', nameCar)
        myCar.insert_one(carQuery)
        print('Đã thêm dữ liệu xe:', nameCar)
        x = myCar.find_one(carQuery)

    # Make request and parse HTML
    try:
        page = requests.get(linkCar)
        soup = BeautifulSoup(page.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        logging.error("Error fetching webpage:", e)
        continue  # Skip to next iteration on fetching error

    # Extract and store tables
    products = soup.select('div.css-vhlprz')
    step = 0
    for product in products:
        # Find the h3 tag within the product
        title = product.select('h3')
        
        bodys = product.select('li')
        step = 0
        # Store extracted data with appropriate structure
        for body in bodys:
            carQuery = {"idCar": x["_id"]}
            myCarFind = myCar.find_one(carQuery)

            if myCarFind:
                print('Dữ liệu đã tồn tại:', nameCar)
                step = 1
                break
            info = {
                "idCar": x["_id"],
                "title": title[0].text,
                "body" : body.text
            }
            myRelatedInfo.insert_one(info)
        print(step)
        if step == 1:
            step = 0
            break
    print('open ', step)

    print('Đã thêm dữ liệu thông số cho xe:', nameCar)

print('Đã hoàn tất thêm dữ liệu vào collection specs')


open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW X5
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW X3
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 5 Series
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 3 Series
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 7 Series
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW 6 Series
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW Z4
open  0
Đã thêm dữ liệu thông số cho xe: 2005 BMW M3
open  0
Đã thêm dữ liệu thông số cho xe: 2006 BMW X5


KeyboardInterrupt: 